<br>
<br>
<br>
<center><big><b>LZ-unmix</center></big><br></b>
<center><b>Hi!</b> Welcome to <i>LZ-unmix</i>, a python package to separate susceptibility components of distorted magnetic hysteresis!<br></center>
<br>
<hr>
&ensp;<b>1.</b> Before running the cells, make sure all of the extensions are corrected installed in your machine<i> (check the READ_ME_1 file);</i><br>
&ensp;<b>2.</b> Make sure your file is included in the same directory holding <i>LZ-unmix</i>;<br>
&ensp;<b>3.</b> Individually run each code cell by firstly clicking on a cell and sequentially clicking on the <i>Run</i> button (or press <i> Shift+Enter</i>);<br>
&ensp;<b>4.</b> Unless you are an advanced python-user, avoid modifications on the code.<br>
&ensp;<b>5.</b> We recommend cleaning the kernel before running a new sample. Go to the Kernel tab and click on <i>"Restart & Clear Output"</i>;<br>
&ensp;<b>6.</b> Images and csv-files from this package will be saved in the same directory.
<hr>
<br>



<br>
<br>
<br>
<hr>
<center><b><big>1. INITIALIZATION</center></b></big>
<hr>
<br>
<br>
<br>

In [ ]:
%matplotlib notebook
import pandas as pd
import math
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from LZ_unmix_functions import f1,f2,f3,ferro_area,region_selection,line_inv,moving_mean,gradient,numerical_int,Ms,Mrs
from LZ_unmix_functions import LV_whithout_plot1,LV_whithout_plot2,LV_whithout_plot3
from LZ_unmix_functions import Levenberg_Marquardt1,Levenberg_Marquardt2,Levenberg_Marquardt3
from IPython.display import display
import seaborn as sns
import scipy.stats
import plotly.graph_objects as go

plt.rcParams.update({'font.size': 10})



<br>
<br>
<br>
<hr>
<center><b><big>2. LOADING THE FILE</center></b></big>
<hr>
<br>
<br>
<br>

In [ ]:
print('Please, use the button select your file! ')
print('Make sure your file has two columns, one for x (field, in Tesla) and other for y (magnetic momentum, Am²), delimited by a comma. ')

uploader = widgets.FileUpload()
display(uploader)


<br>
<br>
<br>
<hr>
<center><b><big>3. FILTERING</center></b></big>
<hr>
<br>
<br>
<br>

In [ ]:
file=uploader.metadata[0]['name']
sample=input('Name your sample! R: ')
grouping = int(input("Enter the grouping value for a moving mean filter (must be an integer>1)! R: "))
rows=int(input("How many rows should be skipped (must be an integer)? R: "))

x,y=np.loadtxt(file,delimiter=',',unpack=True,skiprows=rows)

mass_norm=input('Would you like to normalize by mass?(Y/N) R: ')
if mass_norm=='Y':
    mass=float(input('Provide the mass (kg) R: '))
    y=y/mass
    
    xnew,ynew=moving_mean(x,grouping),moving_mean(y,grouping)

    grad_y=gradient(xnew,ynew)
    grad_original=np.copy(grad_y)


    factor=max(grad_y)
    grad_y=grad_y/factor
    figure,(ax1,ax2)=plt.subplots(1,2,figsize=(9,4))

    ax1.scatter(x,y,marker='.',c='gray')
    ax1.plot(xnew,ynew,marker='.',c='gray',alpha=0.1,label='Smoothing',)
    ax1.legend(shadow=True)
    ax1.set_xlabel(r'$B \ (T)$')
    ax1.set_ylabel(r'$Magnetic \  moment \ (Am²/kg)$')
    ax1.grid(alpha=0.5)


    ax2.scatter(xnew,grad_y,marker='.',color='gray')
    ax2.set_xlabel(r'$B \ (T)$')
    ax2.set_ylabel(r'$\partial{M}_{(M/Mmax)} / \partial{B}$')
    ax2.grid(alpha=0.5)
    figure.tight_layout()

    plt.savefig('Smoothing and First Derivative '+str(sample)+'.pdf',dpi=300,facecolor='w')

if mass_norm=='N':
    y=y
    
    xnew,ynew=moving_mean(x,grouping),moving_mean(y,grouping)

    grad_y=gradient(xnew,ynew)
    grad_original=np.copy(grad_y)


    factor=max(grad_y)
    grad_y=grad_y/factor
    figure,(ax1,ax2)=plt.subplots(1,2,figsize=(9,4))

    ax1.scatter(x,y,marker='.',c='gray')
    ax1.plot(xnew,ynew,marker='.',c='gray',alpha=0.1,label='Smoothing',)
    ax1.legend(shadow=True)
    ax1.set_xlabel(r'$B \ (T)$')
    ax1.set_ylabel(r'$Magnetic \  moment \ (Am²)$')
    ax1.grid(alpha=0.5)


    ax2.scatter(xnew,grad_y,marker='.',color='gray')
    ax2.set_xlabel(r'$B \ (T)$')
    ax2.set_ylabel(r'$\partial{M}_{(M/Mmax)} / \partial{B}$')
    ax2.grid(alpha=0.5)
    figure.tight_layout()

    plt.savefig('Smoothing and First Derivative '+str(sample)+'.pdf',dpi=300,facecolor='w')
    
else:
    print('Invalid, please restart the cell!')


<br>
<br>
<br>
<hr>
<center><b><big>4. DIRECT MODEL</center></b></big>
<hr>
<br>
<br>
<br>

&ensp;<b>A ↓</b> Estimate the scale of the ferromagnetic contribution (A<sub>t</sub>):
<br>

In [ ]:
guesses=ferro_area(xnew,grad_y,region_selection,line_inv,numerical_int)
print(r'The total (normalized) ferromagnetic area (At) is:',float(np.round(guesses,2)))

&ensp;<b>B↓</b> Adjust a direct model with 1, 2 or 3 susceptibility components:
<br>
<br>

In [ ]:
def directmodel1(X0=grad_y[0]/20, Bc1=0.001, W1=0.23, A1=guesses[0]):
    const1=X0
    const2=2/np.pi
    Y=np.zeros(np.size(xnew))
    for i in range(np.size(Y)):
        Y[i]=(const2*((abs(A1)*(abs(W1)/((4*((xnew[i]-abs(Bc1))**2))+(W1**2))))))+const1

    figure,(ax1)=plt.subplots(figsize=(5,5))
    ax1.plot(xnew,Y,label='Direct model',color='brown')
    ax1.scatter(xnew,grad_y,marker='.',label='Data',color='gray',alpha=0.5)
    ax1.set_xlabel(r'$B \ (T)$')
    ax1.set_ylabel(r'$\partial{M} / \partial{B}_{(M/Mmax)} $')
    ax1.set_title(f'{sample}')
    ax1.legend(shadow=True)
    ax1.grid(alpha=0.5)
    figure.tight_layout()
    plt.savefig('Direct model '+str(sample)+'.pdf',dpi=300,facecolor='w')
    parameters=[X0,Bc1,W1,A1]
    return parameters

def directmodel2(X0=grad_y[0]/20, Bc1=0.001, W1=0.23, A1=guesses[0], Bc2=0.37, W2=0.2, A2=guesses[0]):
    const1=X0
    const2=2/np.pi
    Y=np.zeros(np.size(xnew))
    C1=np.zeros(np.size(xnew))
    C2=np.zeros(np.size(xnew))
    for i in range(np.size(Y)):
        Y[i]=(const2*((abs(A1)*(abs(W1)/((4*((xnew[i]-abs(Bc1))**2))+(W1**2))))+(abs(A2)*(abs(W2)/((4*((xnew[i]-abs(Bc2))**2))+(W2**2))))))+const1
        C1[i]=(const2*((abs(A1)*(abs(W1)/((4*((xnew[i]-abs(Bc1))**2))+(W1**2))))))
        C2[i]=(const2*((abs(A2)*(abs(W2)/((4*((xnew[i]-abs(Bc2))**2))+(W2**2))))))
        
    figure,(ax1)=plt.subplots(figsize=(5,5))
    ax1.plot(xnew,Y,label='Direct model',color='brown')
    ax1.plot(xnew,C1,label=r'$C_{a}$',color='royalblue',linestyle='dashed')
    ax1.plot(xnew,C2,label=r'$C_{b}$',color='forestgreen',linestyle='dashed')
    ax1.scatter(xnew,grad_y,marker='.',label='Data',color='gray',alpha=0.5)
    ax1.set_xlabel(r'$B \ (T)$')
    ax1.set_ylabel(r'$\partial{M} / \partial{B}_{(M/Mmax)} $')
    ax1.set_title(f'{sample}')
    ax1.legend(shadow=True)
    ax1.grid(alpha=0.5)
    figure.tight_layout()
    plt.savefig('Direct model '+str(sample)+'.pdf',dpi=300,facecolor='w')
    parameters=[X0,Bc1,W1,A1,Bc2,W2,A2]
    return parameters

def directmodel3(X0=grad_y[0]/20, Bc1=0.001, W1=0.23, A1=guesses[0], Bc2=0.37, W2=0.2, A2=guesses[0], Bc3=0.6, W3=0.2, A3=guesses[0]):
    const1=X0
    const2=2/np.pi
    Y=np.zeros(np.size(xnew))
    C1=np.zeros(np.size(xnew))
    C2=np.zeros(np.size(xnew))
    C3=np.zeros(np.size(xnew))
    for i in range(np.size(Y)):
        Y[i]=(const2*((abs(A1)*(abs(W1)/((4*((xnew[i]-abs(Bc1))**2))+(W1**2))))+(abs(A2)*(abs(W2)/((4*((xnew[i]-abs(Bc2))**2))+(W2**2))))+
                     (abs(A3)*(abs(W3)/((4*((xnew[i]-abs(Bc3))**2))+(W3**2))))))+const1
        
        C1[i]=(const2*((abs(A1)*(abs(W1)/((4*((xnew[i]-abs(Bc1))**2))+(W1**2))))))
        C2[i]=(const2*((abs(A2)*(abs(W2)/((4*((xnew[i]-abs(Bc2))**2))+(W2**2))))))
        C3[i]=(const2*((abs(A3)*(abs(W3)/((4*((xnew[i]-abs(Bc3))**2))+(W3**2))))))
        

    figure,(ax1)=plt.subplots(figsize=(5,5))
    ax1.plot(xnew,Y,label='Direct model',color='brown')
    ax1.plot(xnew,C1,label=r'$C_{a}$',color='royalblue',linestyle='dashed')
    ax1.plot(xnew,C2,label=r'$C_{b}$',color='forestgreen',linestyle='dashed')
    ax1.plot(xnew,C3,label=r'$C_{c}$',color='m',linestyle='dashed')
    ax1.scatter(xnew,grad_y,marker='.',label='Data',color='gray',alpha=0.5)
    ax1.set_xlabel(r'$B \ (T)$')
    ax1.set_ylabel(r'$\partial{M} / \partial{B}_{(M/Mmax)} $')
    ax1.set_title(f'{sample}')
    ax1.legend(shadow=True)
    ax1.grid(alpha=0.5)
    figure.tight_layout()
    plt.savefig('Direct model '+str(sample)+'.pdf',dpi=300,facecolor='w')
    parameters=[X0,Bc1,W1,A1,Bc2,W2,A2,Bc3,W3,A3]
    return parameters

Choice1=input('Would you like to adjust 1, 2 or 3 components (must be an integer)? R: ')

if Choice1=='1':
    ajust=directmodel1
if Choice1=='2':
    ajust=directmodel2
if Choice1=='3':
    ajust=directmodel3




a=widgets.interactive(ajust,X0=(-1,max(grad_y),abs(grad_y[0]/30)), Bc1=(0,2,0.0001),
                 W1=(0,1,0.01), A1=(0,guesses[0]+(2.5*guesses[0]),0.0001),
                      Bc2=(0,2,0.0001), W2=(0,1,0.01),A2=(0,guesses[0]+(2.5*guesses[0]),0.0001),
                      Bc3=(0,2,0.0001), W3=(0,1,0.01),A3=(0,guesses[0]+(2.5*guesses[0]),0.0001));
display(a)



<br>
<br>
<br>
<hr>
<center><b><big>5. INVERSE MODEL</center></b></big>
<hr>
<br>
<br>
<br>

&ensp;<b>A ↓</b> Set the initial guesses for optimization <i>(χ<sub>0</sub>,Bc,W,A)</i>:
<br>

In [ ]:

constrain=input('Would you like to constrain the solutions assuming that Bc is a constant?(Y/N) R: ')
Choice2=input('Would you like to use the parameters adjusted in the direct model as your first guess for the inversion routine(Y/N)? R: ')

if Choice1=='1' and (Choice2=='Y' or Choice2=='y' or Choice2=='yes'):
    y0_direct=a.kwargs.get('X0')*factor
    Bc1_direct=a.kwargs.get('Bc1')
    W1_direct=a.kwargs.get('W1')
    A1_direct=a.kwargs.get('A1')*factor
    
if Choice1=='1' and (Choice2=='N' or Choice2=='n' or Choice2=='no'):
    print('  ')
    print('Provide the inputs for the seven parameters!')
    y0_direct=float(input('Provide y0! R: '))
    Bc1_direct=float(input('Provide Bc1! R: '))
    W1_direct=float(input('Provide W1! R: '))
    A1_direct=float(input('Provide A1! R: '))
    

if Choice1=='2' and (Choice2=='Y' or Choice2=='y' or Choice2=='yes'):
    y0_direct=a.kwargs.get('X0')*factor
    Bc1_direct=a.kwargs.get('Bc1')
    W1_direct=a.kwargs.get('W1')
    A1_direct=a.kwargs.get('A1')*factor
    Bc2_direct=a.kwargs.get('Bc2')
    W2_direct=a.kwargs.get('W2')
    A2_direct=a.kwargs.get('A2')*factor
    
if Choice1=='2' and (Choice2=='N' or Choice2=='n' or Choice2=='no'):
    print('  ')
    print('Provide the inputs for the seven parameters!')
    y0_direct=float(input('Provide y0! R: '))
    Bc1_direct=float(input('Provide Bc1! R: '))
    W1_direct=float(input('Provide W1! R: '))
    A1_direct=float(input('Provide A1! R: '))
    Bc2_direct=float(input('Provide Bc2! R: '))
    W2_direct=float(input('Provide W2! R: '))
    A2_direct=float(input('Provide A2! R: '))
    
    
if Choice1=='3' and (Choice2=='Y' or Choice2=='y' or Choice2=='yes'):
    y0_direct=a.kwargs.get('X0')*factor
    Bc1_direct=a.kwargs.get('Bc1')
    W1_direct=a.kwargs.get('W1')
    A1_direct=a.kwargs.get('A1')*factor
    Bc2_direct=a.kwargs.get('Bc2')
    W2_direct=a.kwargs.get('W2')
    A2_direct=a.kwargs.get('A2')*factor
    Bc3_direct=a.kwargs.get('Bc3')
    W3_direct=a.kwargs.get('W3')
    A3_direct=a.kwargs.get('A3')*factor
    
elif Choice1=='3' and (Choice2=='N' or Choice2=='n' or Choice2=='no'):
    print('  ')
    print('Provide the inputs for the seven parameters!')
    y0_direct=float(input('Provide y0! R: '))
    Bc1_direct=float(input('Provide Bc1! R: '))
    W1_direct=float(input('Provide W1! R: '))
    A1_direct=float(input('Provide A1! R: '))
    Bc2_direct=float(input('Provide Bc2! R: '))
    W2_direct=float(input('Provide W2! R: '))
    A2_direct=float(input('Provide A2! R: '))
    Bc3_direct=float(input('Provide x32! R: '))
    W3_direct=float(input('Provide W3! R: '))
    A3_direct=float(input('Provide A3! R: '))
    


<br>
&ensp;<b>B ↓</b> Define your preferences for the inversion routine. Remember, the convergence criteria (ε) might affect convergence, so you might like to run this step more than once if the final adjust is not satisfying. 
<br>
<br>

In [ ]:
Convergence=float(input('What is the criteria (ε) for convergence? Must be a float, default is 1e-5. R: '))
Iterations=int(input('What is the limit of iterations (must be an integer)?. R: '))
inv=int(input('Define the number of models to be tested (must be an integer)! R: '))

<br>
&ensp;<b>C ↓</b> Create a multidimensional array of initial guesses for optimization, using those set in step <i>5A</i> as the first row of parameters to be inverted, and creating n-others rows with disturbed initial guesses. As the models are sequentially run, <b>this step may take a few seconds</b>.
<br>
<br>


In [ ]:

if Choice1=='1':
    ranged=np.array([y0_direct,Bc1_direct,W1_direct,A1_direct])
    variable_guesses=np.zeros((inv,np.size(ranged)))
    
    if constrain=='Y':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0]*0.5)))
                    variable_guesses[i,1]=np.array(ranged[1])
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2]*0.5)))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3]*0.5)))
    if constrain=='N':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0]*0.5)))
                    variable_guesses[i,1]=np.array(ranged[1]+np.random.normal(ranged[0],abs(ranged[0]*0.5)))
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2]*0.5)))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3]*0.5)))


    v_inv=np.zeros((len(grad_y),inv))
    error_2=np.zeros(inv)
    parameterc_ac=np.zeros((len(ranged),inv))
    for j in range(inv):
        error_2[j],v_inv[:,j],parameterc_ac[:,j]=LV_whithout_plot1(f1,variable_guesses[j,0],variable_guesses[j,1],variable_guesses[j,2],
                                                   variable_guesses[j,3],xnew,grad_original,constrain=constrain,condition=1e-8,index=sample,
                                                            sx=0.00002);




if Choice1=='2':
    ranged=np.array([y0_direct,Bc1_direct,W1_direct,A1_direct,Bc2_direct,W2_direct,A2_direct])
    variable_guesses=np.zeros((inv,np.size(ranged)))
    if constrain=='Y':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0])*0.5))
                    variable_guesses[i,1]=np.array(ranged[1])
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2])*0.5))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3])*0.5))
                    variable_guesses[i,4]=np.array(ranged[4])
                    variable_guesses[i,5]=np.array(ranged[5]+np.random.normal(ranged[5],abs(ranged[5])*0.5))
                    variable_guesses[i,6]=np.array(ranged[6]+np.random.normal(ranged[6],abs(ranged[6])*0.5))
    if constrain=='N':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0])*0.5))
                    variable_guesses[i,1]=np.array(ranged[1]+np.random.normal(ranged[1],abs(ranged[1]*0.5)))
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2]*0.5)))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3]*0.5)))
                    variable_guesses[i,4]=np.array(ranged[4]+np.random.normal(ranged[4],abs(ranged[4]*0.5)))
                    variable_guesses[i,5]=np.array(ranged[5]+np.random.normal(ranged[5],abs(ranged[5]*0.5)))
                    variable_guesses[i,6]=np.array(ranged[6]+np.random.normal(ranged[6],abs(ranged[6]*0.5)))
                    
    v_inv=np.zeros((len(grad_y),inv))
    error_2=np.zeros(inv)
    parameterc_ac=np.zeros((len(ranged),inv))

    for j in range(inv):
        error_2[j],v_inv[:,j],parameterc_ac[:,j]=LV_whithout_plot2(f2,variable_guesses[j,0],variable_guesses[j,1],variable_guesses[j,2],
                                                                   variable_guesses[j,3],variable_guesses[j,4],variable_guesses[j,5],
                                                                   variable_guesses[j,6],xnew,grad_original,f1,constrain,condition=1e-8,index=sample,sx=0.00002);

if Choice1=='3':
    ranged=np.array([y0_direct,Bc1_direct,W1_direct,A1_direct,Bc2_direct,W2_direct,A2_direct,Bc3_direct,W3_direct,A3_direct])
    variable_guesses=np.zeros((inv,np.size(ranged)))
    
    if constrain=='Y':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0])*0.5))
                    variable_guesses[i,1]=np.array(ranged[1])
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2]*0.5)))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3]*0.5)))
                    variable_guesses[i,4]=np.array(ranged[4])
                    variable_guesses[i,5]=np.array(ranged[5]+np.random.normal(ranged[5],abs(ranged[5]*0.5)))
                    variable_guesses[i,6]=np.array(ranged[6]+np.random.normal(ranged[6],abs(ranged[6]*0.5)))
                    variable_guesses[i,7]=np.array(ranged[7])
                    variable_guesses[i,8]=np.array(ranged[8]+np.random.normal(ranged[8],abs(ranged[8]*0.5)))
                    variable_guesses[i,9]=np.array(ranged[9]+np.random.normal(ranged[9],abs(ranged[9]*0.5)))
                    
    if constrain=='N':
        for i in range(inv):
            for j in range(np.size(variable_guesses[1])):
                if j==0:
                    variable_guesses[i,j]=np.array(ranged[j])
                else:
                    variable_guesses[i,0]=np.array(ranged[0]+np.random.normal(ranged[0],abs(ranged[0])*0.5))
                    variable_guesses[i,1]=np.array(ranged[1]+np.random.normal(ranged[1],abs(ranged[1])*0.5))
                    variable_guesses[i,2]=np.array(ranged[2]+np.random.normal(ranged[2],abs(ranged[2])*0.5))
                    variable_guesses[i,3]=np.array(ranged[3]+np.random.normal(ranged[3],abs(ranged[3])*0.5))
                    variable_guesses[i,4]=np.array(ranged[4]+np.random.normal(ranged[4],abs(ranged[4])*0.5))
                    variable_guesses[i,5]=np.array(ranged[5]+np.random.normal(ranged[5],abs(ranged[5])*0.5))
                    variable_guesses[i,6]=np.array(ranged[6]+np.random.normal(ranged[6],abs(ranged[6])*0.5))
                    variable_guesses[i,7]=np.array(ranged[7]+np.random.normal(ranged[7],abs(ranged[7])*0.5))
                    variable_guesses[i,8]=np.array(ranged[8]+np.random.normal(ranged[8],abs(ranged[8])*0.5))
                    variable_guesses[i,9]=np.array(ranged[9]+np.random.normal(ranged[9],abs(ranged[9])*0.5))


    v_inv=np.zeros((len(grad_y),inv))
    error_2=np.zeros(inv)
    parameterc_ac=np.zeros((len(ranged),inv))
    for j in range(inv):
        error_2[j],v_inv[:,j],parameterc_ac[:,j]=LV_whithout_plot3(f3,variable_guesses[j,0],variable_guesses[j,1],variable_guesses[j,2],
                                                   variable_guesses[j,3],variable_guesses[j,4],variable_guesses[j,5],
                                                   variable_guesses[j,6],variable_guesses[j,7],variable_guesses[j,8],
                                                                  variable_guesses[j,9],xnew,grad_original,f1,constrain,condition=1e-8,index=sample,
                                                            sx=0.00002);





<br>
&ensp;<b>D ↓</b> Plot the direct models using the optimized parameters obtained in the previous step and shows an hystogram distribution of their euclidian norms <b>‖e‖</b><sub>2</sub>
<br>
<br>

In [ ]:
figure,(ax4,ax5)=plt.subplots(1,2,figsize=(10,5))
ax4.plot(xnew,v_inv,alpha=0.2)
ax4.scatter(xnew,grad_original,marker='.',color='gray',alpha=0.6,label='Data')
ax4.set_xlabel(r'$B \ (T)$')
ax4.set_ylabel(r'$\partial{M} / \partial{B}_{(M/Mmax)} $')
ax4.legend(shadow=True)
ax4.grid(alpha=0.5)
ax5=sns.histplot(error_2,kde=True,palette=['royalblue'])
ax5.set_xlabel(f'$||e||_{2}$')
ax5.set_ylabel('Counts')
ax5.grid(alpha=0.5)
figure.tight_layout()
plt.savefig('Inversion_models '+str(sample)+'.pdf',dpi=300,facecolor='w')

<br>
&ensp;<b>E ↓</b> Use the parameters that produced the smallest <b>‖e‖</b><sub>2</sub>value to invert a final model.
<br>
<br>

In [ ]:

low_error=np.where(error_2==error_2.min())
low_error=int(low_error[0])

kick=parameterc_ac[:,low_error]

if Choice1=='1':
    p_lm,bubble_deltad_lm,uncertainty_lm,y_inv=Levenberg_Marquardt1(f1,kick[0],kick[1],kick[2],kick[3],xnew,grad_original,constrain=constrain,
                                                                    condition=Convergence,maxiter=Iterations,index=sample,sx=2.0E-5,sy=max(grad_original)*0.05);
if Choice1=='2':
    p_lm,bubble_deltad_lm,uncertainty_lm,y_inv=Levenberg_Marquardt2(f2,kick[0],kick[1],kick[2],kick[3],
                                                             kick[4],kick[5],kick[6],xnew,grad_original,f1,constrain=constrain,
                                                             condition=Convergence,maxiter=Iterations,index=sample,sx=2.0E-5,sy=max(grad_original)*0.05);
if Choice1=='3':
    p_lm,bubble_deltad_lm,uncertainty_lm,y_inv=Levenberg_Marquardt3(f3,kick[0],kick[1],kick[2],kick[3],
                                                             kick[4],kick[5],kick[6],kick[7],kick[8],kick[9],xnew,grad_original,f1,constrain=constrain,condition=Convergence,maxiter=Iterations,index=sample,
                                                        sx=2.0E-5,sy=max(grad_original)*0.05);

<br>
&ensp;<b>F ↓</b> Apply a simple <b>Two-tailed F-test</b> to verify if the variance of the final model can be distinguished from the variance of the data at 95% of confidence.
<br>
<br>

In [ ]:
F_stat=np.var(grad_original,ddof=1)/np.var(y_inv,ddof=1)
df=np.size(grad_original)-1

#find F critical value
critical_f=scipy.stats.f.ppf(q=1-(0.05/2), dfn=df, dfd=df)

if critical_f>F_stat:
    print('Calculated F-value:',F_stat)
    print('Critical F-value:',critical_f)
    print('The variances of the inversion and the data are indistinguishable  using the two-tailed F-test')
if critical_f<F_stat:
    print('Calculated F-value:',F_stat)
    print('Critical F-value:',critical_f)
    print('The variances of the inversion and the data are distinguishable  using the two-tailed F-test')


<br>
&ensp;<b>G ↓</b> Calculate magnetization saturation <b><i>(Ms)</b></i> and magnetization saturation of remanence <i><b>(Mrs)</b></i> through the inverted parameters.
<br>
<br>

In [ ]:
Ms_c=[]
Mrs_c=[]

if Choice1=='1':
    Ms_c=Ms(p_lm[3])
    Mrs_c=Mrs(p_lm[1],p_lm[2],p_lm[3])
    
if Choice1=='2':
    Ms_c=[Ms(p_lm[3]),Ms(p_lm[6])]
    Mrs_c=[Mrs(p_lm[1],p_lm[2],p_lm[3]),Mrs(p_lm[4],p_lm[5],p_lm[6])]
    
if Choice1=='3':
    Ms_c=[Ms(p_lm[3]),Ms(p_lm[6]),Ms(p_lm[9])]
    Mrs_c=[Mrs(p_lm[1],p_lm[2],p_lm[3]),Mrs(p_lm[4],p_lm[5],p_lm[6]),Mrs(p_lm[7],p_lm[8],p_lm[9])]

<br>
&ensp;<b>H ↓</b> Generate tables to display modeling parameters.
<br>
<br>

In [ ]:
if Choice1=='1':
    fig1 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Ca']),
                     cells=dict(values=[['Bc', 'W', 'A','Ms','Mrs'],
                                        [p_lm[1],p_lm[2],p_lm[3],Ms_c,Mrs_c]]))
                         ])
    fig1.update_layout(width=1000, height=400)
    fig1.show()


    fig2 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Values']),
                     cells=dict(values=[['χ0', 'χferro', '||e||2', 'Calculated F-value','Critical F-value'],
                                        [p_lm[0],Ms_c,bubble_deltad_lm,F_stat,critical_f]]))
                         ])
    fig2.update_layout(width=1000, height=400)
    fig2.show()


if Choice1=='2':
    fig1 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Ca', 'Cb']),
                     cells=dict(values=[['Bc', 'W', 'A','Ms','Mrs'],
                                        [p_lm[1],p_lm[2],p_lm[3],Ms_c[0],Mrs_c[0]],
                                       [p_lm[4],p_lm[5],p_lm[6],Ms_c[1],Mrs_c[1]]]))
                         ])
    fig1.update_layout(width=1000, height=400)
    fig1.show()


    fig2 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Values']),
                     cells=dict(values=[['χ0', 'χferro', '||e||2', 'Calculated F-value','Critical F-value'],
                                        [p_lm[0],Ms_c[0]+Ms_c[1],bubble_deltad_lm,F_stat,critical_f]]))
                         ])
    fig2.update_layout(width=1000, height=400)
    fig2.show()
    
if Choice1=='3':
    fig1 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Ca', 'Cb','Cc']),
                     cells=dict(values=[['Bc', 'W', 'A','Ms','Mrs'],
                                        [p_lm[1],p_lm[2],p_lm[3],Ms_c[0],Mrs_c[0]],
                                       [p_lm[4],p_lm[5],p_lm[6],Ms_c[1],Mrs_c[1]],
                                       [p_lm[7],p_lm[8],p_lm[9],Ms_c[2],Mrs_c[2]]]))
                         ])
    fig1.update_layout(width=1000, height=400)
    fig1.show()


    fig2 = go.Figure(data=[go.Table(header=dict(values=['Parameters','Values']),
                     cells=dict(values=[['χ0', 'χferro', '||e||2', 'Calculated F-value','Critical F-value'],
                                        [p_lm[0],Ms_c[0]+Ms_c[1]+Ms_c[2],bubble_deltad_lm,F_stat,critical_f]]))
                         ])
    fig2.update_layout(width=1000, height=400)
    fig2.show()

<br>
<br>
<br>
<hr>
<center><b><big> ACKNOWLEDGEMENTS </center></b></big>
<hr>
<br>
<hr>
&ensp;We would like to thank the creators and collaborators of the python libraries used in this package:
<br>
<br>
<br>
<center>&ensp;Numpy <b>[1]</b>, Matplotlib <b>[2]</b>, Scipy <b>[3]</b>, Pandas <b>[4]</b>, Seaborn <b>[5]</b>, Plotly <b>[6]</b></center>
<br>
<br>

   
&ensp;&ensp;<b>[1]</b> Harris, C.R., and others. Array programming with NumPy. <i>Nature</i> 585, 357–362 (2020). DOI: 10.1038/s41586-020-2649-2.

&ensp;&ensp;<b>[2]</b> J. D. Hunter, "Matplotlib: A 2D Graphics Environment", <i>Computing in Science & Engineering</i>, vol. 9, no. 3, pp. 90-95, 2007. DOI: 10.5281/zenodo.592536.

&ensp;&ensp;<b>[3]</b> Virtanen, P., and others (2020) SciPy 1.0: Fundamental Algorithms for Scientific Computing in Python. <i>Nature Methods</i>, 17(3), 261-272. DOI: 10.1038/s41592-019-0686-2.

&ensp;&ensp;<b>[4]</b> McKinney, W., and others. (2010). Data structures for statistical computing in python. In: <i>Proceedings of the 9th Python in Science Conference</i> (Vol. 445, pp. 51–56).

&ensp;&ensp;<b>[5]</b> Waskom, M., and others (2017). mwaskom/seaborn: v0.8.1 (September 2017). Zenodo. DOI: 10.5281/zenodo.883859

&ensp;&ensp;<b>[6]</b> Inc., P. T. (2015). Collaborative data science. Montreal, QC: Plotly Technologies Inc. Retrieved from https://plot.ly.

<hr>